# Library


In [3]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import pyexasol

In [4]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# Folder

In [5]:
df1_fold = 'D:/data/big2/BD1/df'

# Server

In [6]:
# # insider db
# wd = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
# we = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='vsyse')

In [7]:
# exasol db
ws = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYS')
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='vsyse')

# Load

## 등록정보(STD_CEG_CAR_MIG) 4등급만

In [8]:
# exasol db
# 14.2s
car = wd.export_to_pandas("SELECT VHRNO, VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO FROM STD_CEG_CAR_MIG WHERE EXHST_GAS_GRD_CD = 'A0504' OR EXHST_GAS_GRD_CD = 'A05T4';")

In [9]:
car_ch_col = {
    'VHRNO':'자동차등록번호', 
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자',
    'VHCL_MNG_NO':'차량관리번호', 
}

In [10]:
carr = car.rename(columns=car_ch_col)

In [11]:
carr.columns

Index(['자동차등록번호', '차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호',
       '차량연식', '차종', '용도', '최초등록일자', '제작일자', '차량관리번호'],
      dtype='object')

## 제원정보(STD_CEG_CAR_SRC_MIG)

In [12]:
# 3.8s
src = wd.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY FROM STD_CEG_CAR_SRC_MIG;")

In [13]:
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
}

In [14]:
srcr = src.rename(columns=src_ch_col)
srcr.columns

Index(['제원관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식'], dtype='object')

## 저감장치 부착이력(STD_DLM_TB_ERP_ATT_HIS)

In [15]:
# 3.3s
att = wd.export_to_pandas("SELECT VIN, RDCDVC_SE_CD, RDCDVC_KND_CD FROM STD_DLM_TB_ERP_ATT_HIS;")
att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010946 entries, 0 to 1010945
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   VIN            1010946 non-null  object
 1   RDCDVC_SE_CD   1010946 non-null  object
 2   RDCDVC_KND_CD  756468 non-null   object
dtypes: object(3)
memory usage: 23.1+ MB


In [16]:
att_ch_col = {
    'VIN':'차대번호', 
    'RDCDVC_SE_CD':'저감장치구분', 
    'RDCDVC_KND_CD':'저감장치종류', 
}

In [17]:
attr = att.rename(columns=att_ch_col)
attr.columns

Index(['차대번호', '저감장치구분', '저감장치종류'], dtype='object')

## 노후차 조기폐차 관리정보(수도권)(STD_DLM_TB_ERP_EARLY_ERASE_AEA)

In [18]:
# 2.4s
aea = wd.export_to_pandas("SELECT VIN, ELPDSRC_STTS_CD, ELPDSRC_LST_APRV_YN FROM STD_DLM_TB_ERP_EARLY_ERASE_AEA;")
aea.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804082 entries, 0 to 804081
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   VIN                  804082 non-null  object
 1   ELPDSRC_STTS_CD      804082 non-null  object
 2   ELPDSRC_LST_APRV_YN  683874 non-null  object
dtypes: object(3)
memory usage: 18.4+ MB


In [19]:
aea.columns

Index(['VIN', 'ELPDSRC_STTS_CD', 'ELPDSRC_LST_APRV_YN'], dtype='object')

In [20]:
aea_ch_col = {
    'VIN':'차대번호', 
    'ELPDSRC_STTS_CD':'조기폐차상태코드', 
    'ELPDSRC_LST_APRV_YN':'조기폐차최종승인YN',  
}

In [21]:
aear = aea.rename(columns=aea_ch_col)
aear.columns

Index(['차대번호', '조기폐차상태코드', '조기폐차최종승인YN'], dtype='object')

In [22]:
aear.head()

,차대번호,조기폐차상태코드,조기폐차최종승인YN
0,KMCXLS7BP2U626435,A32Y,Y
1,KNAJA5265VA794691,A32Y,Y
2,KPBLC3D81VP018295,A32Y,Y
3,KPDHADMD1YP225493,A32Y,Y
4,KNAMA75122S249083,A32Y,Y


## 노후차 조기폐차 관리정보(수도권외)(STD_DLM_TB_ERP_EARLY_ERASE_LGV)

In [23]:
# 1.8s
lgv = wd.export_to_pandas("SELECT VIN, ELPDSRC_STTS_CD, ELPDSRC_LST_APRV_YN FROM STD_DLM_TB_ERP_EARLY_ERASE_LGV;")
lgv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373371 entries, 0 to 373370
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   VIN                  373371 non-null  object
 1   ELPDSRC_STTS_CD      373371 non-null  object
 2   ELPDSRC_LST_APRV_YN  160690 non-null  object
dtypes: object(3)
memory usage: 8.5+ MB


In [24]:
lgv.columns

Index(['VIN', 'ELPDSRC_STTS_CD', 'ELPDSRC_LST_APRV_YN'], dtype='object')

In [25]:
lgv_ch_col = {
    'VIN':'차대번호', 
    'ELPDSRC_STTS_CD':'조기폐차상태코드', 
    'ELPDSRC_LST_APRV_YN':'조기폐차최종승인YN',  
}

In [26]:
lgvr = lgv.rename(columns=lgv_ch_col)
lgvr.columns

Index(['차대번호', '조기폐차상태코드', '조기폐차최종승인YN'], dtype='object')

In [27]:
lgvr.head()

,차대번호,조기폐차상태코드,조기폐차최종승인YN
0,KMHSD81VP4U791871,A32M,NaN
1,KPBFA2AF16P223800,A32M,NaN
2,KMFWVH7HP7U772891,A32M,NaN
3,KMFXKN7BPYU361960,A32M,NaN
4,KMFXKS7BP2U563724,A32M,NaN


## 법정동코드(STD_BJCD_INFO)

In [28]:
# 1.3s
code = wd.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM FROM STD_BJCD_INFO;")
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47916 entries, 0 to 47915
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STDG_CD       47916 non-null  int64 
 1   STDG_CTPV_NM  47916 non-null  object
 2   STDG_SGG_NM   47892 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [29]:
code.columns

Index(['STDG_CD', 'STDG_CTPV_NM', 'STDG_SGG_NM'], dtype='object')

In [30]:
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구',  
}

In [31]:
coder = code.rename(columns=code_ch_col)
coder.columns

Index(['법정동코드', '시도', '시군구'], dtype='object')

In [32]:
coder.head()

,법정동코드,시도,시군구
0,1100000000,서울특별시,NaN
1,1111000000,서울특별시,종로구
2,1111010100,서울특별시,종로구
3,1111010200,서울특별시,종로구
4,1111010300,서울특별시,종로구


## 등록이력(CEG_CAR_HISTORY_MIG)

In [33]:
# 1m 13.2s
his = ws.export_to_pandas("SELECT VHCL_ERSR_YN, CHNG_DE, VHMNO FROM CEG_CAR_HISTORY_MIG;")
his_ch_col = {
    'VHCL_ERSR_YN':'차량말소YN', 
    'CHNG_DE':'변경일자',
    'VHMNO':'차량관리번호'
}
hisr = his.rename(columns=his_ch_col)
hisr.columns

Index(['차량말소YN', '변경일자', '차량관리번호'], dtype='object')

In [34]:
# # 1.8s
# edb_id = 'vsysd'
# edb_database = 'edb'
# edb_port = 5444
# edb_url = '172.29.135.50'
# edb_pwd = 'vsyswynn'
# conn = psycopg2.connect(dbname=edb_database, user=edb_id, password=edb_pwd, host=edb_url, port=edb_port)
# cur = conn.cursor()
# sql = 'select VHCL_ERSR_YN, CHNG_DE, VHMNO from vsysd.ceg_car_history_mig'
# cur.execute(sql)
# his = pd.DataFrame(cur.fetchall())
# his.columns = [desc[0].upper() for desc in cur.description]
# cur.close()
# conn.close()

## RH에서 제공한 법정동코드

In [35]:
rh = we.export_to_pandas("SELECT DONG_CODE, CTPRVN_NM, SIGNGU_NM FROM TB_MAPDATA;")
rh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   DONG_CODE  278 non-null    int64 
 1   CTPRVN_NM  278 non-null    object
 2   SIGNGU_NM  260 non-null    object
dtypes: int64(1), object(2)
memory usage: 6.6+ KB


In [36]:
rh = rh.rename(columns={'DONG_CODE':'법정동코드_rh', 'CTPRVN_NM':'시도', 'SIGNGU_NM':'시군구'})
rh.head()

,법정동코드_rh,시도,시군구
0,0,전국,NaN
1,1100000000,서울특별시,NaN
2,1111000000,서울특별시,종로구
3,1114000000,서울특별시,중구
4,1117000000,서울특별시,용산구


In [37]:
rh.shape

(278, 3)

## 4등급 result(for DPF유무)

In [38]:
rs = we.export_to_pandas("SELECT 차대번호, DPF유무_수정 FROM GRD4_RESULT;")
rs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502235 entries, 0 to 1502234
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   차대번호      1502235 non-null  object
 1   DPF유무_수정  1153813 non-null  object
dtypes: object(2)
memory usage: 22.9+ MB


# 전처리

## 등록정보(CEG_CAR_MIG)

### 중복 차대번호 제거

In [39]:
carr.shape, len(carr['차대번호'].unique())

((1898814, 13), 1898763)

In [40]:
carr['최초등록일자'] = pd.to_numeric(carr['최초등록일자'], errors='coerce')
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

(1898763, 13)

### 차종 코드 변환

In [41]:
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
carr['차종'].unique()

array(['화물', '승합', '승용', '특수', nan], dtype=object)

### 용도 코드 변환

In [42]:
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
carr['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

### 등록정보 말소 제거

In [43]:
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

(1515736, 13)

## 제원정보(CEG_CAR_SRC_MIG)

### 연료 코드 변환

In [44]:
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['경유', '휘발유', 'CNG(압축천연가스)', nan, 'LPG(액화석유가스)', '기타연료',
       '휘발유 하이브리드', '전기', '알코올', '등유', '태양열', 'CNG 하이브리드', '경유 하이브리드',
       'LPG 하이브리드', '수소', 'LNG(액화천연가스)'], dtype=object)

### 차종유형 코드 변환

In [45]:
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '소형', '중형', '경형', nan], dtype=object)

## 등록&제원 병합

In [46]:
# 19.4s
cs = carm.merge(srcr, on='제원관리번호', how='left')
cs.shape

(1515736, 19)

## 저감장치 부착이력(DLM_TB_ERP_ATT_HIS)

### 저감장치구분 코드 변환

In [47]:
attr['저감장치구분'].unique()

array(['A1001', 'A1003', 'A1005', 'A1006', 'A1002', 'A1004', 'A1007'],
      dtype=object)

In [48]:
# 27.5s
rdcdvc_dict = {
    'A1001':'1종', 
    'A1002':'2종', 
    'A1003':'3종', 
    'A1004':'1종+SCR', 
    'A1005':'엔진개조', 
    'A1006':'엔진교체',
    'A1007':'삼원촉매',
}
attr['저감장치구분'] = attr['저감장치구분'].replace(rdcdvc_dict)
attr['저감장치구분'].unique()

array(['1종', '3종', '엔진개조', '엔진교체', '2종', '1종+SCR', '삼원촉매'], dtype=object)

### 저감장치종류 코드 변환

In [49]:
attr['저감장치종류'].unique()

array(['A1313', 'A1311', nan, 'A1309', 'A1307', 'A1306', 'A1305', 'A1303',
       'A1310', 'A1308', 'A1302', 'A1304', 'A1312'], dtype=object)

In [50]:
rdcdvc_knd_dict = {
    'A1301':'dPDF',
    'A1302':'대형',
    'A1303':'복합대형',
    'A1304':'복합대형+SCR',
    'A1305':'복합소형',
    'A1306':'복합중형',
    'A1307':'소형',
    'A1308':'자연대형+SCR',
    'A1309':'자연중형',
    'A1310':'중형',
    'A1311':'DOC',
    'A1312':'자연소형',
    'A1313':'자연대형',
}
attr['저감장치종류'] = attr['저감장치종류'].replace(rdcdvc_knd_dict)
attr['저감장치종류'].unique()

array(['자연대형', 'DOC', nan, '자연중형', '소형', '복합중형', '복합소형', '복합대형', '중형',
       '자연대형+SCR', '대형', '복합대형+SCR', '자연소형'], dtype=object)

### 저감장치 부착 유무

In [51]:
attr.loc[(attr['저감장치구분'] == '1종') | (attr['저감장치구분'] == '1종+SCR'), 'DPF_YN'] = '유'

In [52]:
attr['DPF_YN'].value_counts(dropna=False)

DPF_YN
NaN    531833
유      479113
Name: count, dtype: int64

## 노후차 조기폐차(STD_DLM_TB_ERP_EARLY_ERASE_AEA, LGV)

### 조기폐차상태코드 코드 변환

In [53]:
aear['조기폐차상태코드'].unique()

array(['A32Y', 'A32G', 'A32B', 'A32M', 'A32C', 'A32X', 'A32A', 'A32D',
       'A32N', 'A32P', 'A32I'], dtype=object)

In [54]:
erase_dict = {
    'A32E':'조기폐차상태코드(추가보조금신청대상)',
    'A32G':'조기폐차상태코드(보조금청구)',
    'A32I':'조기폐차상태코드(신청등록)',
    'A32K':'조기폐차상태코드(추가보조금청구승인)',
    'A32M':'조기폐차상태코드(보조금산정)',
    'A32N':'조기폐차상태코드(보조금청구반려(제외))',
    'A32P':'조기폐차상태코드(보조금대상)',
    'A32T':'조기폐차상태코드(추가보조금청구)',
    'A32X':'조기폐차상태코드(신청취소(제외))',
    'A32Y':'조기폐차상태코드(보조금청구승인)',
    'A32C':'조기폐차상태코드(성능확인검사등록)',
    'A32D':'조기폐차상태코드(기간초과)',
    'A32A':'조기폐차상태코드(성능확인검사신청)',
    'A32B':'조기폐차상태코드(보조금미대상)',
}
aear['조기폐차상태코드'] = aear['조기폐차상태코드'].replace(erase_dict)
aear['조기폐차상태코드'].unique()

array(['조기폐차상태코드(보조금청구승인)', '조기폐차상태코드(보조금청구)', '조기폐차상태코드(보조금미대상)',
       '조기폐차상태코드(보조금산정)', '조기폐차상태코드(성능확인검사등록)', '조기폐차상태코드(신청취소(제외))',
       '조기폐차상태코드(성능확인검사신청)', '조기폐차상태코드(기간초과)', '조기폐차상태코드(보조금청구반려(제외))',
       '조기폐차상태코드(보조금대상)', '조기폐차상태코드(신청등록)'], dtype=object)

In [55]:
lgvr['조기폐차상태코드'].unique()

array(['A32M', 'A32X', 'A32C', 'A32B', 'A32Y', 'A32G', 'A32D', 'A32P',
       'A32I', 'A32N'], dtype=object)

In [56]:
lgvr['조기폐차상태코드'] = lgvr['조기폐차상태코드'].replace(erase_dict)
lgvr['조기폐차상태코드'].unique()

array(['조기폐차상태코드(보조금산정)', '조기폐차상태코드(신청취소(제외))', '조기폐차상태코드(성능확인검사등록)',
       '조기폐차상태코드(보조금미대상)', '조기폐차상태코드(보조금청구승인)', '조기폐차상태코드(보조금청구)',
       '조기폐차상태코드(기간초과)', '조기폐차상태코드(보조금대상)', '조기폐차상태코드(신청등록)',
       '조기폐차상태코드(보조금청구반려(제외))'], dtype=object)

### 조기폐차 신청 정보 추가

In [57]:
aear['조기폐차신청여부'] = 'Y'
lgvr['조기폐차신청여부'] = 'Y'

## 등록&제원&저감

In [58]:
cs.shape

(1515736, 19)

In [59]:
csa = cs.merge(attr, on='차대번호', how='left')
csa.shape

(1515736, 22)

## 등록&제원&저감&법정동

In [60]:
coder.columns

Index(['법정동코드', '시도', '시군구'], dtype='object')

In [61]:
csac = csa.merge(coder, on='법정동코드', how='left')
csac.shape

(1515736, 24)

In [62]:
csac['시도'].isnull().sum()

1

In [63]:
# 경기도 양주시
csac.loc[csac['법정동코드'] == 4163055000, ['시도', '시군구']] = ['경기도', '양주시']

In [64]:
csac['시도'].isnull().sum()

0

## 조기폐차 병합

In [65]:
elp = pd.concat([aear, lgvr], ignore_index=True)
elp.shape

(1177453, 4)

In [66]:
elp.shape, len(elp['차대번호'].unique())

((1177453, 4), 1105195)

In [67]:
elpm = elp.sort_values('조기폐차최종승인YN', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
elpm.shape

(1105195, 4)

In [68]:
elpm = elpm[elpm['조기폐차최종승인YN'] == 'Y'].reset_index(drop=True)
elpm.shape

(844474, 4)

## 등록&제원&저감&법정동&조기폐차

In [69]:
csac.shape

(1515736, 24)

In [70]:
df = csac.merge(elpm, on='차대번호', how='left')
df.shape

(1515736, 27)

In [71]:
df['연료'].value_counts(dropna=False)

연료
경유             1156239
휘발유             333206
LPG(액화석유가스)      26179
NaN                 71
기타연료                39
CNG(압축천연가스)          1
알코올                  1
Name: count, dtype: int64

## RH법정동 코드 참고하여 수정

In [72]:
df.columns

Index(['자동차등록번호', '차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호',
       '차량연식', '차종', '용도', '최초등록일자', '제작일자', '차량관리번호', '연료', '차종유형', '제작사명',
       '차명', '자동차형식', '엔진형식', '저감장치구분', '저감장치종류', 'DPF_YN', '시도', '시군구',
       '조기폐차상태코드', '조기폐차최종승인YN', '조기폐차신청여부'],
      dtype='object')

In [73]:
rdf = df.copy()

In [74]:
rdf['법정동코드'] = rdf['법정동코드'].astype('str')
rdf['법정동코드_mod'] = rdf['법정동코드'].str[:5] + '00000'
rdf['법정동코드_mod'] = pd.to_numeric(rdf['법정동코드_mod'])
rdf['법정동코드_mod'].head()

0    4824000000
1    4482500000
2    4518000000
3    2826000000
4    2771000000
Name: 법정동코드_mod, dtype: int64

In [75]:
set(rdf['법정동코드_mod'].unique()) - set(rh['법정동코드_rh'].unique())

{2817000000, 4119500000, 4119700000, 4119900000, 4173000000, 4371000000}

In [76]:
# 2817000000 인천광역시 남구 -> 인천광역시 미추홀구 2817700000
# 4119500000 경기도 부천시 원미구 -> 경기도 부천시 4119000000
# 4119700000 경기도 부천시 소사구 -> 경기도 부천시 4119000000
# 4119900000 경기도 부천시 오정구 -> 경기도 부천시 4119000000
# 4173000000 경기도 여주군 -> 경기도 여주시 4167000000
# 4371000000 충청북도 청원군 -> 충청북도 충주시 4311000000

In [77]:
rdf.loc[rdf['법정동코드_mod'] == 2817000000, '법정동코드_mod'] = 2817700000
rdf.loc[rdf['법정동코드_mod'] == 4119500000, '법정동코드_mod'] = 4119000000
rdf.loc[rdf['법정동코드_mod'] == 4119700000, '법정동코드_mod'] = 4119000000
rdf.loc[rdf['법정동코드_mod'] == 4119900000, '법정동코드_mod'] = 4119000000
rdf.loc[rdf['법정동코드_mod'] == 4173000000, '법정동코드_mod'] = 4167000000
rdf.loc[rdf['법정동코드_mod'] == 4371000000, '법정동코드_mod'] = 4311000000

In [78]:
set(rdf['법정동코드_mod'].unique()) - set(rh['법정동코드_rh'].unique())

set()

In [79]:
rdf.columns

Index(['자동차등록번호', '차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호',
       '차량연식', '차종', '용도', '최초등록일자', '제작일자', '차량관리번호', '연료', '차종유형', '제작사명',
       '차명', '자동차형식', '엔진형식', '저감장치구분', '저감장치종류', 'DPF_YN', '시도', '시군구',
       '조기폐차상태코드', '조기폐차최종승인YN', '조기폐차신청여부', '법정동코드_mod'],
      dtype='object')

In [80]:
rdf['법정동코드_mod'].isnull().sum()

0

In [81]:
df = rdf.copy()
df.columns

Index(['자동차등록번호', '차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호',
       '차량연식', '차종', '용도', '최초등록일자', '제작일자', '차량관리번호', '연료', '차종유형', '제작사명',
       '차명', '자동차형식', '엔진형식', '저감장치구분', '저감장치종류', 'DPF_YN', '시도', '시군구',
       '조기폐차상태코드', '조기폐차최종승인YN', '조기폐차신청여부', '법정동코드_mod'],
      dtype='object')

## 4등급 result 파일 참고하여 DPF유무 수정

In [82]:
rdf = df.copy()

In [83]:
len(set(rdf['차대번호'].unique()) - set(rs['차대번호'].unique()))

53185

In [84]:
rdf['DPF_YN'].value_counts(dropna=False)

DPF_YN
NaN    1515675
유           61
Name: count, dtype: int64

In [85]:
rs['DPF유무_수정'].value_counts(dropna=False)

DPF유무_수정
무       879481
NaN     348422
유       261897
확인불가     12435
Name: count, dtype: int64

In [86]:
rs = rs.drop_duplicates('차대번호').reset_index(drop=True)
rs.shape

(1502213, 2)

In [87]:
rs['DPF유무_수정'].value_counts(dropna=False)

DPF유무_수정
무       879480
NaN     348401
유       261897
확인불가     12435
Name: count, dtype: int64

In [88]:
rdf1 = rdf.merge(rs, on='차대번호', how='left')

In [89]:
rdf1.loc[(rdf1['DPF_YN'] == '유') | (rdf1['DPF유무_수정'] == '유'), 'DPF_YN'] = '유'
rdf1.loc[(rdf1['DPF유무_수정'] == '무'), 'DPF_YN'] = '무'
rdf1.loc[(rdf1['DPF유무_수정'] == '확인불가'), 'DPF_YN'] = '확인불가'

In [90]:
rdf1['DPF_YN'].value_counts(dropna=False)

DPF_YN
무       839835
NaN     401576
유       261892
확인불가     12433
Name: count, dtype: int64

In [91]:
rdf1.columns

Index(['자동차등록번호', '차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호',
       '차량연식', '차종', '용도', '최초등록일자', '제작일자', '차량관리번호', '연료', '차종유형', '제작사명',
       '차명', '자동차형식', '엔진형식', '저감장치구분', '저감장치종류', 'DPF_YN', '시도', '시군구',
       '조기폐차상태코드', '조기폐차최종승인YN', '조기폐차신청여부', '법정동코드_mod', 'DPF유무_수정'],
      dtype='object')

In [92]:
df = rdf1.drop('DPF유무_수정', axis=1)

In [93]:
df.shape

(1515736, 28)

## 말소차량 데이터

### 등록 & 제원 정보 병합(말소 유지)

In [94]:
carr.shape

(1898763, 13)

In [95]:
csersr = carr.merge(srcr[['제원관리번호', '연료']], on='제원관리번호', how='left')
csersr.shape

(1898763, 14)

### 1\. 차량관리번호 기준 병합

In [96]:
csersr.shape

(1898763, 14)

In [97]:
# 58.3s
ersr = csersr.merge(hisr, on='차량관리번호', how='left')
ersr.shape

(4063735, 16)

### 2\. 차량말소YN 만 추출

In [98]:
errm = ersr[(ersr['차량말소YN_x'] == 'Y') & (ersr['차량말소YN_y'] == 'Y')].reset_index(drop=True)
errm.shape

(369842, 16)

### 3\. 변경일자 최신으로 차대번호 중복 제거

In [99]:
errm = errm.sort_values('변경일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)

### 4\. 변경일자 2019.01.01 이상만 추출

In [100]:
errm = errm[errm['변경일자'] >= 20190101].reset_index(drop=True)

In [101]:
errm.shape

(367090, 16)

In [102]:
errm.columns

Index(['자동차등록번호', '차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN_x', '제원관리번호',
       '차량연식', '차종', '용도', '최초등록일자', '제작일자', '차량관리번호', '연료', '차량말소YN_y',
       '변경일자'],
      dtype='object')

### 5\. 법정동 코드 정보 병합

In [103]:
errc = errm.merge(coder, on='법정동코드', how='left')
errc.shape

(367090, 18)

# 분석

## 4등급 경유 조기폐차

In [104]:
df1 = df[df['연료'] == '경유'].reset_index(drop=True)
df1.shape

(1156239, 28)

In [105]:
df1.columns

Index(['자동차등록번호', '차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호',
       '차량연식', '차종', '용도', '최초등록일자', '제작일자', '차량관리번호', '연료', '차종유형', '제작사명',
       '차명', '자동차형식', '엔진형식', '저감장치구분', '저감장치종류', 'DPF_YN', '시도', '시군구',
       '조기폐차상태코드', '조기폐차최종승인YN', '조기폐차신청여부', '법정동코드_mod'],
      dtype='object')

In [106]:
STD_BD_GRD4_ELPDSRC_CURSTT = df1[[
    '차대번호', 
    '법정동코드', 
    '차종', 
    '용도', 
    '연료', 
    '차종유형', 
    '시도',
    '시군구', 
    '조기폐차상태코드', 
    '조기폐차최종승인YN',
]]

In [107]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20230717'

In [108]:
STD_BD_GRD4_ELPDSRC_CURSTT['테이블생성일자'] = today_date

C:\Users\kbjung\AppData\Local\Temp\ipykernel_6940\2309398753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  STD_BD_GRD4_ELPDSRC_CURSTT['테이블생성일자'] = today_date


In [109]:
STD_BD_GRD4_ELPDSRC_CURSTT = STD_BD_GRD4_ELPDSRC_CURSTT[[
    '테이블생성일자',  
    '차대번호', 
    '법정동코드', 
    '차종', 
    '용도', 
    '연료', 
    '차종유형', 
    '시도', 
    '시군구', 
    '조기폐차상태코드',
    '조기폐차최종승인YN', 
]]

In [110]:
chc_dict = {
    '테이블생성일자':'LOAD_DT', 
    '기준연월':'CRTR_YM', 
    '차대번호':'VIN', 
    '법정동코드':'STDG_CD', 
    '차종':'VHCTY_CD', 
    '용도':'PURPS_CD2', 
    '연료':'FUEL_CD', 
    '차종유형':'VHCTY_TY', 
    '시도':'CTPV', 
    '시군구':'SGG', 
    '조기폐차상태코드':'ELPDSRC_STTS_CD',
    '조기폐차최종승인YN':'ELPDSRC_LST_APRV_YN', 
}

In [111]:
STD_BD_GRD4_ELPDSRC_CURSTT = STD_BD_GRD4_ELPDSRC_CURSTT.rename(columns=chc_dict)
STD_BD_GRD4_ELPDSRC_CURSTT.columns

Index(['LOAD_DT', 'VIN', 'STDG_CD', 'VHCTY_CD', 'PURPS_CD2', 'FUEL_CD',
       'VHCTY_TY', 'CTPV', 'SGG', 'ELPDSRC_STTS_CD', 'ELPDSRC_LST_APRV_YN'],
      dtype='object')

In [112]:
STD_BD_GRD4_ELPDSRC_CURSTT.head()

,LOAD_DT,VIN,STDG_CD,VHCTY_CD,PURPS_CD2,FUEL_CD,VHCTY_TY,CTPV,SGG,ELPDSRC_STTS_CD,ELPDSRC_LST_APRV_YN
0,20230717,KMCLK18KP8C035195,4824034000,화물,개인용,경유,대형,경상남도,사천시,NaN,NaN
1,20230717,KMFLA18KP8C031274,4482533000,화물,개인용,경유,대형,충청남도,태안군,NaN,NaN
2,20230717,KMCGK17HP9C112030,4518010400,화물,개인용,경유,중형,전라북도,정읍시,NaN,NaN
3,20230717,KLACA26RD8B226753,2826011300,승용,개인용,경유,중형,인천광역시,서구,NaN,NaN
4,20230717,KNCSE01429K388374,2771026522,화물,개인용,경유,소형,대구광역시,달성군,NaN,NaN


### [출력] STD_BD_GRD4_ELPDSRC_CURSTT

In [113]:
# expdf = STD_BD_GRD4_ELPDSRC_CURSTT
# table_nm = 'STD_BD_GRD4_ELPDSRC_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

In [114]:
# # 
# STD_BD_GRD4_ELPDSRC_CURSTT.to_csv(os.path.join(df1_fold, 'STD_BD_GRD4_ELPDSRC_CURSTT.csv'), index=False)

## 4등급 세분류

In [115]:
df['연료'].value_counts(dropna=False)

연료
경유             1156239
휘발유             333206
LPG(액화석유가스)      26179
NaN                 71
기타연료                39
CNG(압축천연가스)          1
알코올                  1
Name: count, dtype: int64

### fuel 컬럼 추가

In [116]:
df1 = df.copy()

In [117]:
df1.loc[df1['연료'] == '경유', 'fuel'] = '경유'
df1.loc[(df1['연료'] == '휘발유') | (df1['연료'] == 'LPG(액화석유가스)'), 'fuel'] = '휘발유_가스'

In [118]:
df1['fuel'].value_counts(dropna=False)

fuel
경유        1156239
휘발유_가스     359385
NaN           112
Name: count, dtype: int64

### EG 분류

In [119]:
grade_list = []
for f, y, cy, e in tqdm(df1[['fuel', '제작일자', '차량연식', 'DPF_YN']].values):
    if (f == '휘발유_가스') and ( (19980101 <= y <= 20001231) or (1998 <= cy <= 2000) ):
        grade_list.append('A')
    elif (f == '휘발유_가스') and ( (y <= 19971231) or (cy <= 1997) ):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '유'):
        grade_list.append('A')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) )and (e == '유'):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '무'):
        grade_list.append('C')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) ) and (e == '무'):
        grade_list.append('D')
    else:
        grade_list.append('X')
len(grade_list)

  0%|          | 0/1515736 [00:00<?, ?it/s]

100%|██████████| 1515736/1515736 [00:02<00:00, 523104.43it/s]


1515736

In [120]:
df1['Grade'] = grade_list

In [121]:
df1.columns

Index(['자동차등록번호', '차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호',
       '차량연식', '차종', '용도', '최초등록일자', '제작일자', '차량관리번호', '연료', '차종유형', '제작사명',
       '차명', '자동차형식', '엔진형식', '저감장치구분', '저감장치종류', 'DPF_YN', '시도', '시군구',
       '조기폐차상태코드', '조기폐차최종승인YN', '조기폐차신청여부', '법정동코드_mod', 'fuel', 'Grade'],
      dtype='object')

In [122]:
STD_BD_GRD4_MLSFC_RSLT = df1[[
    '차대번호', 
    '제원관리번호',
    '차종', 
    '용도', 
    '차량연식', 
    '차종유형', 
    '연료', 
    '법정동코드', 
    '시도', 
    '시군구', 
    'DPF_YN',
    'Grade',
    '법정동코드_mod',
    ]]
STD_BD_GRD4_MLSFC_RSLT.columns

Index(['차대번호', '제원관리번호', '차종', '용도', '차량연식', '차종유형', '연료', '법정동코드', '시도',
       '시군구', 'DPF_YN', 'Grade', '법정동코드_mod'],
      dtype='object')

In [123]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20230717'

In [124]:
STD_BD_GRD4_MLSFC_RSLT['테이블생성일자'] = today_date

C:\Users\kbjung\AppData\Local\Temp\ipykernel_6940\792058024.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  STD_BD_GRD4_MLSFC_RSLT['테이블생성일자'] = today_date


In [125]:
STD_BD_GRD4_MLSFC_RSLT = STD_BD_GRD4_MLSFC_RSLT[[
    '테이블생성일자', 
    '차대번호', 
    '제원관리번호', 
    '차종', 
    '용도', 
    '차량연식', 
    '차종유형', 
    '연료', 
    '법정동코드', 
    '시도',
    '시군구', 
    'DPF_YN', 
    'Grade', 
    '법정동코드_mod',
    ]]
STD_BD_GRD4_MLSFC_RSLT.columns

Index(['테이블생성일자', '차대번호', '제원관리번호', '차종', '용도', '차량연식', '차종유형', '연료', '법정동코드',
       '시도', '시군구', 'DPF_YN', 'Grade', '법정동코드_mod'],
      dtype='object')

In [126]:
STD_BD_GRD4_MLSFC_RSLT['Grade'].value_counts(dropna=False)

Grade
C    800869
B    387925
A    215684
X     72292
D     38966
Name: count, dtype: int64

In [127]:
STD_BD_GRD4_MLSFC_RSLT['법정동코드_mod'] = STD_BD_GRD4_MLSFC_RSLT['법정동코드_mod'].astype('str')

In [128]:
ch_col_dict = {
                '테이블생성일자':'LOAD_DT',
                '차대번호':'VIN', 
                '제원관리번호':'MANG_MNG_NO',
                '차종':'VHCTY_CD', 
                '용도':'PURPS_CD2',
                '차량연식':'YRIDNW', 
                '차종유형':'VHCTY_TY', 
                '연료':'FUEL_CD', 
                '법정동코드':'STDG_CD', 
                '시도':'CTPV_NM',
                '시군구':'SGG_NM',
                'DPF_YN':'DPF_MNTNG_YN',
                'Grade':'GRD4_MLSFC', 
                '법정동코드_mod':'STDG_CD_MOD'
                }

In [129]:
STD_BD_GRD4_MLSFC_RSLT = STD_BD_GRD4_MLSFC_RSLT.rename(columns=ch_col_dict)
STD_BD_GRD4_MLSFC_RSLT.columns

Index(['LOAD_DT', 'VIN', 'MANG_MNG_NO', 'VHCTY_CD', 'PURPS_CD2', 'YRIDNW',
       'VHCTY_TY', 'FUEL_CD', 'STDG_CD', 'CTPV_NM', 'SGG_NM', 'DPF_MNTNG_YN',
       'GRD4_MLSFC', 'STDG_CD_MOD'],
      dtype='object')

In [130]:
STD_BD_GRD4_MLSFC_RSLT.tail()

,LOAD_DT,VIN,MANG_MNG_NO,VHCTY_CD,PURPS_CD2,YRIDNW,VHCTY_TY,FUEL_CD,STDG_CD,CTPV_NM,SGG_NM,DPF_MNTNG_YN,GRD4_MLSFC,STDG_CD_MOD
1515731,20230717,KNAD2322JS055503,10015800000000,승용,개인용,1995,소형,휘발유,4155025029,경기도,안성시,NaN,B,4155000000
1515732,20230717,KNADA2322HS006936,10015800000000,승용,개인용,1987,소형,휘발유,5011013700,제주특별자치도,제주시,NaN,B,5011000000
1515733,20230717,2T1CF2897YC359923,99931107024351303,승용,개인용,2000,대형,휘발유,1117012400,서울특별시,용산구,NaN,A,1117000000
1515734,20230717,SL68T032460,10017500140000,승용,개인용,1968,대형,휘발유,1165010100,서울특별시,서초구,NaN,B,1165000000
1515735,20230717,CN-62-T-2213,30110100002636,승용,개인용,2002,소형,휘발유,1159010200,서울특별시,동작구,NaN,X,1159000000


### [출력] STD_BD_GRD4_MLSFC_RSLT

In [131]:
# expdf = STD_BD_GRD4_MLSFC_RSLT
# table_nm = 'STD_BD_GRD4_MLSFC_RSLT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)

In [132]:
# # 
# STD_BD_GRD4_MLSFC_RSLT.to_csv(os.path.join(df1_fold, 'STD_BD_GRD4_MLSFC_RSLT.csv'), index=False)

## 4등급 등급세분류

In [253]:
df1.columns

Index(['자동차등록번호', '차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호',
       '차량연식', '차종', '용도', '최초등록일자', '제작일자', '차량관리번호', '연료', '차종유형', '제작사명',
       '차명', '자동차형식', '엔진형식', '저감장치구분', '저감장치종류', 'DPF_YN', '시도', '시군구',
       '조기폐차상태코드', '조기폐차최종승인YN', '조기폐차신청여부', '법정동코드_mod', 'fuel', 'Grade'],
      dtype='object')

In [254]:
dat_mlsfc = df1.copy()

In [255]:
dat_mlsfc['시군구_수정'] = dat_mlsfc['시군구'].str.split(' ').str[0]

In [256]:
dat_mlsfc['Grade'].isnull().sum()

0

In [269]:
grp1 = dat_mlsfc.groupby(['연료', '시도', '시군구_수정', '차종', '차종유형', '용도', 'Grade'])['차대번호'].count().unstack('Grade').reset_index()
grp1

Grade,연료,시도,시군구_수정,차종,차종유형,용도,A,B,C,D,X
0,CNG(압축천연가스),전라남도,신안군,화물,대형,개인용,NaN,NaN,NaN,NaN,1.00
1,LPG(액화석유가스),강원도,강릉시,승용,대형,개인용,1.00,NaN,NaN,NaN,1.00
2,LPG(액화석유가스),강원도,강릉시,승용,중형,개인용,9.00,9.00,NaN,NaN,NaN
3,LPG(액화석유가스),강원도,강릉시,승용,중형,영업용,NaN,3.00,NaN,NaN,NaN
4,LPG(액화석유가스),강원도,강릉시,승합,경형,개인용,NaN,3.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
9117,휘발유,충청북도,충주시,승합,경형,개인용,NaN,13.00,NaN,NaN,NaN
9118,휘발유,충청북도,충주시,승합,소형,개인용,NaN,3.00,NaN,NaN,NaN
9119,휘발유,충청북도,충주시,승합,중형,개인용,1.00,1.00,NaN,NaN,NaN
9120,휘발유,충청북도,충주시,화물,경형,개인용,NaN,8.00,NaN,NaN,NaN


In [274]:
# 연도 설정
grp1['연도'] = '2022'
# grp1['연도'] = today_date[:4]
grp1['테이블생성일자'] = today_date

In [275]:
grp1.columns

Index(['연료', '시도', '시군구_수정', '차종', '차종유형', '용도', 'A', 'B', 'C', 'D', 'X', '연도',
       '테이블생성일자'],
      dtype='object', name='Grade')

In [276]:
STD_BD_DAT_GRD4_MLSFC = grp1[[
    '연도', 
    '연료', 
    '시도', 
    '시군구_수정', 
    '차종', 
    '차종유형', 
    '용도', 
    'A', 
    'B', 
    'C', 
    'D', 
    'X',
    '테이블생성일자',
]]

In [277]:
cdict = {
    '연도':'YR', 
    '연료':'FUEL_CD', 
    '시도':'CTPV', 
    '시군구_수정':'SGG', 
    '차종':'VHCTY_CD', 
    '차종유형':'VHCTY_TY', 
    '용도':'PURPS_CD2', 
    'A':'A_MKCNT', 
    'B':'B_MKCNT', 
    'C':'C_MKCNT', 
    'D':'D_MKCNT', 
    'X':'X_MKCNT', 
    '테이블생성일자':'LOAD_DT', 
}
STD_BD_DAT_GRD4_MLSFC = grp1.rename(columns=cdict)
STD_BD_DAT_GRD4_MLSFC.columns

Index(['FUEL_CD', 'CTPV', 'SGG', 'VHCTY_CD', 'VHCTY_TY', 'PURPS_CD2',
       'A_MKCNT', 'B_MKCNT', 'C_MKCNT', 'D_MKCNT', 'X_MKCNT', 'YR', 'LOAD_DT'],
      dtype='object', name='Grade')

### [출력] STD_BD_DAT_GRD4_MLSFC

In [ ]:
# expdf = STD_BD_DAT_GRD4_MLSFC
# table_nm = 'STD_BD_DAT_GRD4_MLSFC'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)

In [278]:
# 
STD_BD_DAT_GRD4_MLSFC.to_csv(os.path.join(df1_fold, 'STD_BD_DAT_GRD4_MLSFC.csv'), index=False)

## 4등급차량 상세정보

In [133]:
carr.shape

(1898763, 13)

In [134]:
cst = carr.merge(srcr, on='제원관리번호', how='left')
cst.shape

(1898763, 19)

In [135]:
csat = cst.merge(attr, on='차대번호', how='left')
csat.shape

(1898763, 22)

In [136]:
csact = csat.merge(coder, on='법정동코드', how='left')
csact.shape

(1898763, 24)

In [137]:
csact['시도'].isnull().sum()

1

In [138]:
csact.loc[csact['시도'].isnull() == True, '법정동코드']

1782457    4163055000
Name: 법정동코드, dtype: int64

In [139]:
# 경기도 양주시
csact.loc[csact['법정동코드'] == 4163055000, ['시도', '시군구']] = ['경기도', '양주시']

In [140]:
csact['시도'].isnull().sum()

0

In [141]:
csact.shape

(1898763, 24)

In [142]:
dft = csact.merge(elpm, on='차대번호', how='left')
dft.shape

(1898763, 27)

### 4등급 result 파일 참고하여 DPF유무 수정

In [143]:
rdf = dft.copy()

In [144]:
len(set(rdf['차대번호'].unique()) - set(rs['차대번호'].unique()))

436196

In [145]:
rdf['DPF_YN'].value_counts(dropna=False)

DPF_YN
NaN    1898683
유           80
Name: count, dtype: int64

In [146]:
rs['DPF유무_수정'].value_counts(dropna=False)

DPF유무_수정
무       879480
NaN     348401
유       261897
확인불가     12435
Name: count, dtype: int64

In [147]:
rs = rs.drop_duplicates('차대번호').reset_index(drop=True)
rs.shape

(1502213, 2)

In [148]:
rs['DPF유무_수정'].value_counts(dropna=False)

DPF유무_수정
무       879480
NaN     348401
유       261897
확인불가     12435
Name: count, dtype: int64

In [149]:
rdf1 = rdf.merge(rs, on='차대번호', how='left')

In [150]:
rdf1.loc[(rdf1['DPF_YN'] == '유') | (rdf1['DPF유무_수정'] == '유'), 'DPF_YN'] = '유'
rdf1.loc[(rdf1['DPF유무_수정'] == '무'), 'DPF_YN'] = '무'
rdf1.loc[(rdf1['DPF유무_수정'] == '확인불가'), 'DPF_YN'] = '확인불가'

In [151]:
rdf1['DPF_YN'].value_counts(dropna=False)

DPF_YN
무       839842
NaN     784572
유       261915
확인불가     12434
Name: count, dtype: int64

In [152]:
rdf1.columns

Index(['자동차등록번호', '차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호',
       '차량연식', '차종', '용도', '최초등록일자', '제작일자', '차량관리번호', '연료', '차종유형', '제작사명',
       '차명', '자동차형식', '엔진형식', '저감장치구분', '저감장치종류', 'DPF_YN', '시도', '시군구',
       '조기폐차상태코드', '조기폐차최종승인YN', '조기폐차신청여부', 'DPF유무_수정'],
      dtype='object')

In [153]:
dft = rdf1.drop('DPF유무_수정', axis=1)

In [154]:
dft.shape

(1898763, 27)

In [155]:
dft.shape, len(dft['차대번호'].unique())

((1898763, 27), 1898763)

In [156]:
dfte = dft.merge(errc[['차대번호', '변경일자']], on='차대번호', how='left')
dfte.shape

(1898763, 28)

In [157]:
df1.columns

Index(['자동차등록번호', '차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호',
       '차량연식', '차종', '용도', '최초등록일자', '제작일자', '차량관리번호', '연료', '차종유형', '제작사명',
       '차명', '자동차형식', '엔진형식', '저감장치구분', '저감장치종류', 'DPF_YN', '시도', '시군구',
       '조기폐차상태코드', '조기폐차최종승인YN', '조기폐차신청여부', '법정동코드_mod', 'fuel', 'Grade'],
      dtype='object')

In [158]:
df1.shape, len(df1['차대번호'].unique())

((1515736, 30), 1515736)

In [159]:
dftem = dfte.merge(df1[['차대번호', 'Grade']], on='차대번호', how='left')
dftem.shape

(1898763, 29)

In [160]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20230717'

In [161]:
dftem['테이블생성일자'] = today_date

In [162]:
list(dftem.columns)

['자동차등록번호',
 '차대번호',
 '법정동코드',
 '배출가스등급',
 '배출가스인증번호',
 '차량말소YN',
 '제원관리번호',
 '차량연식',
 '차종',
 '용도',
 '최초등록일자',
 '제작일자',
 '차량관리번호',
 '연료',
 '차종유형',
 '제작사명',
 '차명',
 '자동차형식',
 '엔진형식',
 '저감장치구분',
 '저감장치종류',
 'DPF_YN',
 '시도',
 '시군구',
 '조기폐차상태코드',
 '조기폐차최종승인YN',
 '조기폐차신청여부',
 '변경일자',
 'Grade',
 '테이블생성일자']

In [242]:
STD_BD_DAT_GRD4_DTL_INFO = dftem[[
    '자동차등록번호',
    '차대번호',
    'Grade',
    '차종',
    '차종유형',
    '용도',
    '연료',
    '시도',
    '시군구',
    '차량연식',
    'DPF_YN',
    '저감장치종류',
    '최초등록일자',
    '조기폐차신청여부',
    '조기폐차상태코드',
    '변경일자',
    '차량말소YN',
    '테이블생성일자', 
    # '법정동코드',
    # '배출가스등급',
    # '배출가스인증번호',
    # '제원관리번호',
    # '제작일자',
    # '차량관리번호',
    # '제작사명',
    # '차명',
    # '자동차형식',
    # '엔진형식',
    # '저감장치구분',
    # '조기폐차최종승인YN',
]]
STD_BD_DAT_GRD4_DTL_INFO.shape

(1898763, 18)

In [243]:
cdict = {
    '자동차등록번호':'VHRNO',
    '차대번호':'VIN',
    'Grade':'GRD4_MLSFC',
    '차종':'VHCTY_CD',
    '차종유형':'VHCTY_TY',
    '용도':'PURPS_CD2',
    '연료':'FUEL_CD',
    '시도':'CTPV',
    '시군구':'SGG',
    '차량연식':'YRIDNW',
    'DPF_YN':'DPF_MNTNG_YN',
    '저감장치종류':'RDCDVC_KND',
    '최초등록일자':'FRST_REG_YMD',
    '조기폐차신청여부':'ELPDSRC_APLY_YN',
    '조기폐차상태코드':'ELPDSRC_STTS_CD',
    '변경일자':'CHNG_DE',
    '차량말소YN':'VHCL_ERSR_YN',
    '테이블생성일자':'LOAD_DT', 
}
STD_BD_DAT_GRD4_DTL_INFO = STD_BD_DAT_GRD4_DTL_INFO.rename(columns=cdict)
STD_BD_DAT_GRD4_DTL_INFO.columns

Index(['VHRNO', 'VIN', 'GRD4_MLSFC', 'VHCTY_CD', 'VHCTY_TY', 'PURPS_CD2',
       'FUEL_CD', 'CTPV', 'SGG', 'YRIDNW', 'DPF_MNTNG_YN', 'RDCDVC_KND',
       'FRST_REG_YMD', 'ELPDSRC_APLY_YN', 'ELPDSRC_STTS_CD', 'CHNG_DE',
       'VHCL_ERSR_YN', 'LOAD_DT'],
      dtype='object')

In [244]:
list(STD_BD_DAT_GRD4_DTL_INFO.columns)

['VHRNO',
 'VIN',
 'GRD4_MLSFC',
 'VHCTY_CD',
 'VHCTY_TY',
 'PURPS_CD2',
 'FUEL_CD',
 'CTPV',
 'SGG',
 'YRIDNW',
 'DPF_MNTNG_YN',
 'RDCDVC_KND',
 'FRST_REG_YMD',
 'ELPDSRC_APLY_YN',
 'ELPDSRC_STTS_CD',
 'CHNG_DE',
 'VHCL_ERSR_YN',
 'LOAD_DT']

In [247]:
STD_BD_DAT_GRD4_DTL_INFO.shape

(1898763, 18)

### [출력] STD_BD_DAT_GRD4_DTL_INFO

In [245]:
# expdf = STD_BD_DAT_GRD4_DTL_INFO
# table_nm = 'STD_BD_DAT_GRD4_DTL_INFO'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 7s
# we.import_from_pandas(expdf, table_nm)

In [246]:
# 9.1s
STD_BD_DAT_GRD4_DTL_INFO.to_csv(os.path.join(df1_fold, 'STD_BD_DAT_GRD4_DTL_INFO.csv'), index=False)

## 4등급 연월, 시도, 시군구별 차량 대수

In [166]:
df['최초등록일자'].describe()

count    1515736.00
mean    20056364.15
std        66402.95
min     19650209.00
25%     20060622.00
50%     20081111.00
75%     20100510.00
max     20220614.00
Name: 최초등록일자, dtype: float64

In [167]:
dfm = df.copy()

In [168]:
dfm['최초등록일자'] = dfm['최초등록일자'].astype('str')

In [169]:
dfm['최초등록일자_년'] = dfm['최초등록일자'].str[:4]
dfm['최초등록일자_월'] = dfm['최초등록일자'].str[4:6]
dfm['최초등록일자_일'] = dfm['최초등록일자'].str[6:8]

### 시군구명 앞쪽 지역명만 남기기(dfm)

In [170]:
# 시군구명 앞쪽 지역명만 남기기(dfm)
dfm['시군구_수정'] = dfm['시군구'].str.split(' ').str[0]

### 현재 연료 지역별 차량대수

In [171]:
dfm['최초등록일자'].max()

'20220614'

In [172]:
num_car_by_local1 = dfm.groupby(['연료', '시도', '시군구_수정'], dropna=False)['차대번호'].count().reset_index()
num_car_by_local1 = num_car_by_local1.rename(columns={'차대번호':'차량대수'})
num_car_by_local1

,연료,시도,시군구_수정,차량대수
0,CNG(압축천연가스),전라남도,신안군,1
1,LPG(액화석유가스),강원도,강릉시,67
2,LPG(액화석유가스),강원도,고성군,15
3,LPG(액화석유가스),강원도,동해시,184
4,LPG(액화석유가스),강원도,삼척시,21
...,...,...,...,...
814,NaN,전라북도,군산시,3
815,NaN,전라북도,익산시,1
816,NaN,충청북도,옥천군,1
817,NaN,충청북도,진천군,1


In [173]:
max_date = dfm['최초등록일자'].max()
max_year = max_date[:4]
max_month = max_date[4:6]
max_year, max_month

('2022', '06')

In [174]:
num_car_by_local1[['연도', '월']] = [max_year, max_month]
num_car_by_local1

,연료,시도,시군구_수정,차량대수,연도,월
0,CNG(압축천연가스),전라남도,신안군,1,2022,06
1,LPG(액화석유가스),강원도,강릉시,67,2022,06
2,LPG(액화석유가스),강원도,고성군,15,2022,06
3,LPG(액화석유가스),강원도,동해시,184,2022,06
4,LPG(액화석유가스),강원도,삼척시,21,2022,06
...,...,...,...,...,...,...
814,NaN,전라북도,군산시,3,2022,06
815,NaN,전라북도,익산시,1,2022,06
816,NaN,충청북도,옥천군,1,2022,06
817,NaN,충청북도,진천군,1,2022,06


### 연료 지역별 등록차량대수

In [175]:
num_car_by_local2 = dfm.groupby(['연료', '시도', '시군구_수정', '최초등록일자_년', '최초등록일자_월'], as_index=False)['차대번호'].count()
num_car_by_local2 = num_car_by_local2.rename(columns={'차대번호':'등록차량대수', '최초등록일자_년':'연도', '최초등록일자_월':'월'})
num_car_by_local2

,연료,시도,시군구_수정,연도,월,등록차량대수
0,CNG(압축천연가스),전라남도,신안군,2005,04,1
1,LPG(액화석유가스),강원도,강릉시,1990,03,1
2,LPG(액화석유가스),강원도,강릉시,1992,04,1
3,LPG(액화석유가스),강원도,강릉시,1993,05,1
4,LPG(액화석유가스),강원도,강릉시,1993,07,1
...,...,...,...,...,...,...
75669,휘발유,충청북도,충주시,2009,02,1
75670,휘발유,충청북도,충주시,2009,03,1
75671,휘발유,충청북도,충주시,2010,03,1
75672,휘발유,충청북도,충주시,2013,05,1


### 연월, 연료 지역별 말소 대수

In [176]:
errc['변경일자'] = errc['변경일자'].astype('str')

In [177]:
errc['변경일자_년'] = errc['변경일자'].str[:4]
errc['변경일자_월'] = errc['변경일자'].str[4:6]
errc['변경일자_일'] = errc['변경일자'].str[6:8]

### 시군구명 앞쪽 지역명만 남기기(errc)

In [178]:
# 시군구명 앞쪽 지역명만 남기기(errc)
errc['시군구_수정'] = errc['시군구'].str.split(' ').str[0]

In [179]:
grp_erase = errc.loc[errc['변경일자_년'] == dfm['최초등록일자_년'].max()].groupby(['변경일자_년', '변경일자_월', '연료', '시도', '시군구_수정'], as_index=False)['차대번호'].count()
grp_erase = grp_erase.rename(columns={'차대번호':'말소차량대수', '변경일자_년':'연도', '변경일자_월':'월'})
grp_erase = grp_erase.sort_values(['시도', '시군구_수정'])
grp_erase

,연도,월,연료,시도,시군구_수정,말소차량대수
0,2022,01,LPG(액화석유가스),강원도,강릉시,1
106,2022,01,경유,강원도,강릉시,26
353,2022,01,휘발유,강원도,강릉시,11
671,2022,02,경유,강원도,강릉시,38
917,2022,02,휘발유,강원도,강릉시,6
...,...,...,...,...,...,...
2256,2022,04,휘발유,충청북도,충주시,6
2609,2022,05,경유,충청북도,충주시,47
2829,2022,05,휘발유,충청북도,충주시,7
3168,2022,06,경유,충청북도,충주시,44


In [180]:
num_car_by_local1['연도'].max()

'2022'

In [181]:
list(pd.date_range(end=dfm['최초등록일자'].max(), periods=4, freq="MS").month)

[3, 4, 5, 6]

In [182]:
y_plist = list(pd.date_range(end=dfm['최초등록일자'].max(), periods=4, freq="MS").year)
mth_plist = list(pd.date_range(end=dfm['최초등록일자'].max(), periods=4, freq="MS").month)
y_plist, mth_plist

([2022, 2022, 2022, 2022], [3, 4, 5, 6])

In [183]:
yr_list = []
mth_list = []
fuel_list = []
ctpv_list = []
sgg_list = []
sl = num_car_by_local1.drop_duplicates(['시도', '시군구_수정']).reset_index(drop=True)
for ctpv, sgg in sl[['시도', '시군구_수정']].values:
    for fuel in sl['연료'].unique():
        for yr, mth in zip(y_plist, mth_plist):
            mthm = f'{mth:0>2}'
            yr_list.append(str(yr))
            mth_list.append(mthm)
            fuel_list.append(fuel)
            ctpv_list.append(ctpv)
            sgg_list.append(sgg)
base = pd.DataFrame({'연도':yr_list, '월':mth_list, '연료':fuel_list, '시도':ctpv_list, '시군구_수정':sgg_list})

In [184]:
base

,연도,월,연료,시도,시군구_수정
0,2022,03,CNG(압축천연가스),전라남도,신안군
1,2022,04,CNG(압축천연가스),전라남도,신안군
2,2022,05,CNG(압축천연가스),전라남도,신안군
3,2022,06,CNG(압축천연가스),전라남도,신안군
4,2022,03,LPG(액화석유가스),전라남도,신안군
...,...,...,...,...,...
4139,2022,06,경유,인천광역시,남구
4140,2022,03,휘발유,인천광역시,남구
4141,2022,04,휘발유,인천광역시,남구
4142,2022,05,휘발유,인천광역시,남구


In [185]:
num_car_by_local1.head()

,연료,시도,시군구_수정,차량대수,연도,월
0,CNG(압축천연가스),전라남도,신안군,1,2022,06
1,LPG(액화석유가스),강원도,강릉시,67,2022,06
2,LPG(액화석유가스),강원도,고성군,15,2022,06
3,LPG(액화석유가스),강원도,동해시,184,2022,06
4,LPG(액화석유가스),강원도,삼척시,21,2022,06


In [186]:
base1 = base.merge(num_car_by_local1, on=['연도', '월', '연료', '시도', '시군구_수정'], how='left')

In [187]:
base2 = base1.merge(num_car_by_local2, on=['연도', '월', '연료', '시도', '시군구_수정'], how='left')

In [188]:
base3 = base2.merge(grp_erase, on=['연도', '월', '연료', '시도', '시군구_수정'], how='left')

In [189]:
base3[['차량대수', '등록차량대수', '말소차량대수']] = base3[['차량대수', '등록차량대수', '말소차량대수']].fillna(0)
base3[['차량대수', '등록차량대수', '말소차량대수']].isnull().sum()

차량대수      0
등록차량대수    0
말소차량대수    0
dtype: int64

In [190]:
n = len(base3['월'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소차량대수'] - base3.loc[(i+1)*n - (j-1), '등록차량대수']

In [191]:
base3

,연도,월,연료,시도,시군구_수정,차량대수,등록차량대수,말소차량대수
0,2022,03,CNG(압축천연가스),전라남도,신안군,1.00,0.00,0.00
1,2022,04,CNG(압축천연가스),전라남도,신안군,1.00,0.00,0.00
2,2022,05,CNG(압축천연가스),전라남도,신안군,1.00,0.00,0.00
3,2022,06,CNG(압축천연가스),전라남도,신안군,1.00,0.00,0.00
4,2022,03,LPG(액화석유가스),전라남도,신안군,23.00,0.00,0.00
...,...,...,...,...,...,...,...,...
4139,2022,06,경유,인천광역시,남구,0.00,0.00,0.00
4140,2022,03,휘발유,인천광역시,남구,10.00,0.00,0.00
4141,2022,04,휘발유,인천광역시,남구,10.00,0.00,0.00
4142,2022,05,휘발유,인천광역시,남구,10.00,0.00,0.00


In [192]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20230717'

In [193]:
base3['테이블생성일자'] = today_date

In [194]:
base4 = base3[[
    '테이블생성일자', 
    '연도',
    '월', 
    '연료', 
    '시도', 
    '시군구_수정', 
    '차량대수',
]]

In [195]:
chc_col = {
    '테이블생성일자':'LOAD_DT', 
    '기준연월':'CRTR_YM',
    '연도':'YR', 
    '월':'MM', 
    '연료':'FUEL_CD', 
    '시도':'CTPV', 
    '시군구_수정':'SGG', 
    '차량대수':'VHCL_MKCNT', 
}

In [196]:
STD_BD_GRD4_RGN_CURSTT = base4.rename(columns=chc_col)
STD_BD_GRD4_RGN_CURSTT.columns

Index(['LOAD_DT', 'YR', 'MM', 'FUEL_CD', 'CTPV', 'SGG', 'VHCL_MKCNT'], dtype='object')

### [출력] STD_BD_GRD4_RGN_CURSTT

In [197]:
# expdf = STD_BD_GRD4_RGN_CURSTT
# table_nm = 'STD_BD_GRD4_RGN_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

In [198]:
# # 0s
# STD_BD_GRD4_RGN_CURSTT.to_csv(os.path.join(df1_fold, 'STD_BD_GRD4_RGN_CURSTT.csv'), index=False)

## 4등급 연도, 시도, 차종별 차량 대수

### 현재 차량 대수

In [199]:
num_car_by_local1 = dfm.groupby(['시도', '차종'], dropna=False)['차대번호'].count().reset_index()
num_car_by_local1 = num_car_by_local1.rename(columns={'차대번호':'차량대수'})
num_car_by_local1

,시도,차종,차량대수
0,강원도,승용,28111
1,강원도,승합,5329
2,강원도,특수,195
3,강원도,화물,22841
4,경기도,승용,215871
...,...,...,...
64,충청남도,화물,35247
65,충청북도,승용,31711
66,충청북도,승합,5920
67,충청북도,특수,300


In [200]:
num_car_by_local1['연도'] = dfm['최초등록일자_년'].max()
num_car_by_local1

,시도,차종,차량대수,연도
0,강원도,승용,28111,2022
1,강원도,승합,5329,2022
2,강원도,특수,195,2022
3,강원도,화물,22841,2022
4,경기도,승용,215871,2022
...,...,...,...,...
64,충청남도,화물,35247,2022
65,충청북도,승용,31711,2022
66,충청북도,승합,5920,2022
67,충청북도,특수,300,2022


### 등록 차량 대수

In [201]:
num_car_by_local2 = dfm.groupby(['시도', '차종', '최초등록일자_년'], as_index=False)['차대번호'].count()
num_car_by_local2 = num_car_by_local2.rename(columns={'차대번호':'등록차량대수', '최초등록일자_년':'연도'})
num_car_by_local2

,시도,차종,연도,등록차량대수
0,강원도,승용,1987,21
1,강원도,승용,1988,253
2,강원도,승용,1989,475
3,강원도,승용,1990,618
4,강원도,승용,1991,743
...,...,...,...,...
1615,충청북도,화물,2016,6
1616,충청북도,화물,2017,12
1617,충청북도,화물,2018,6
1618,충청북도,화물,2019,3


### 말소 차량 대수

In [202]:
grp_erase = errc.groupby(['변경일자_년', '시도', '차종'], as_index=False)['차대번호'].count()
grp_erase = grp_erase.rename(columns={'차대번호':'말소차량대수', '변경일자_년':'연도'})
grp_erase = grp_erase.sort_values(['시도'])
grp_erase

,연도,시도,차종,말소차량대수
0,2019,강원도,승용,1826
203,2022,강원도,승용,1329
206,2022,강원도,화물,602
138,2021,강원도,화물,1185
137,2021,강원도,특수,16
...,...,...,...,...
65,2019,충청북도,특수,13
64,2019,충청북도,승합,263
63,2019,충청북도,승용,1790
202,2021,충청북도,화물,1149


In [203]:
y_plist = list(pd.date_range(end=dfm['최초등록일자'].max(), periods=4, freq="YS").year)
y_plist

[2019, 2020, 2021, 2022]

In [204]:
yr_list = []
fuel_list = []
ctpv_list = []
cd_list = []
for ctpv in num_car_by_local1['시도'].unique():
    for cd in ['승용', '승합', '화물', '특수']:
        for yrm in y_plist:
            yr_list.append(str(yrm))
            fuel_list.append(fuel)
            ctpv_list.append(ctpv)
            cd_list.append(cd)
base = pd.DataFrame({'연도':yr_list, '시도':ctpv_list, '차종':cd_list})

In [205]:
base

,연도,시도,차종
0,2019,강원도,승용
1,2020,강원도,승용
2,2021,강원도,승용
3,2022,강원도,승용
4,2019,강원도,승합
...,...,...,...
267,2022,충청북도,화물
268,2019,충청북도,특수
269,2020,충청북도,특수
270,2021,충청북도,특수


In [206]:
base1 = base.merge(num_car_by_local1, on=['연도', '시도', '차종'], how='left')
base1

,연도,시도,차종,차량대수
0,2019,강원도,승용,NaN
1,2020,강원도,승용,NaN
2,2021,강원도,승용,NaN
3,2022,강원도,승용,28111.00
4,2019,강원도,승합,NaN
...,...,...,...,...
267,2022,충청북도,화물,23262.00
268,2019,충청북도,특수,NaN
269,2020,충청북도,특수,NaN
270,2021,충청북도,특수,NaN


In [207]:
base2 = base1.merge(num_car_by_local2, on=['연도', '시도', '차종'], how='left')
base2

,연도,시도,차종,차량대수,등록차량대수
0,2019,강원도,승용,NaN,2.00
1,2020,강원도,승용,NaN,NaN
2,2021,강원도,승용,NaN,NaN
3,2022,강원도,승용,28111.00,NaN
4,2019,강원도,승합,NaN,1.00
...,...,...,...,...,...
267,2022,충청북도,화물,23262.00,NaN
268,2019,충청북도,특수,NaN,1.00
269,2020,충청북도,특수,NaN,NaN
270,2021,충청북도,특수,NaN,NaN


In [208]:
base3 = base2.merge(grp_erase, on=['연도', '시도', '차종'], how='left')
base3

,연도,시도,차종,차량대수,등록차량대수,말소차량대수
0,2019,강원도,승용,NaN,2.00,1826.00
1,2020,강원도,승용,NaN,NaN,3181.00
2,2021,강원도,승용,NaN,NaN,2974.00
3,2022,강원도,승용,28111.00,NaN,1329.00
4,2019,강원도,승합,NaN,1.00,241.00
...,...,...,...,...,...,...
267,2022,충청북도,화물,23262.00,NaN,571.00
268,2019,충청북도,특수,NaN,1.00,13.00
269,2020,충청북도,특수,NaN,NaN,10.00
270,2021,충청북도,특수,NaN,NaN,17.00


In [209]:
base3[['차량대수', '등록차량대수', '말소차량대수']] = base3[['차량대수', '등록차량대수', '말소차량대수']].fillna(0)
base3[['차량대수', '등록차량대수', '말소차량대수']].isnull().sum()

차량대수      0
등록차량대수    0
말소차량대수    0
dtype: int64

In [210]:
n = len(base3['연도'].unique())
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소차량대수'] - base3.loc[(i+1)*n - (j-1), '등록차량대수']

In [211]:
base3.tail(20)

,연도,시도,차종,차량대수,등록차량대수,말소차량대수
252,2019,충청남도,특수,445.00,1.00,5.00
253,2020,충청남도,특수,438.00,0.00,7.00
254,2021,충청남도,특수,423.00,0.00,15.00
255,2022,충청남도,특수,421.00,0.00,2.00
256,2019,충청북도,승용,39522.00,1.00,1790.00
257,2020,충청북도,승용,36342.00,0.00,3180.00
258,2021,충청북도,승용,33104.00,0.00,3238.00
259,2022,충청북도,승용,31711.00,0.00,1393.00
260,2019,충청북도,승합,6975.00,0.00,263.00
261,2020,충청북도,승합,6585.00,3.00,393.00


In [212]:
today_date = datetime.today().strftime("%Y%m")
today_date

'202307'

In [213]:
base3['테이블생성일자'] = today_date

In [214]:
base4 = base3[[
    '테이블생성일자', 
    '연도', 
    '시도', 
    '차종', 
    '차량대수', 
]]

In [215]:
chc_col = {
    '테이블생성일자':'LOAD_DT',
    '연도':'YR', 
    '시도':'CTPV', 
    '차종':'VHCTY_CD', 
    '차량대수':'VHCL_MKCNT', 
}

In [216]:
STD_BD_GRD4_RGN_CURSTT_MOD = base4.rename(columns=chc_col)
STD_BD_GRD4_RGN_CURSTT_MOD.columns

Index(['LOAD_DT', 'YR', 'CTPV', 'VHCTY_CD', 'VHCL_MKCNT'], dtype='object')

In [217]:
STD_BD_GRD4_RGN_CURSTT_MOD.head()

,LOAD_DT,YR,CTPV,VHCTY_CD,VHCL_MKCNT
0,202307,2019,강원도,승용,35595.00
1,202307,2020,강원도,승용,32414.00
2,202307,2021,강원도,승용,29440.00
3,202307,2022,강원도,승용,28111.00
4,202307,2019,강원도,승합,6391.00


### [출력] STD_BD_GRD4_RGN_CURSTT_MOD

In [218]:
# expdf = STD_BD_GRD4_RGN_CURSTT_MOD
# table_nm = 'STD_BD_GRD4_RGN_CURSTT_MOD'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

In [219]:
# # 0s
# STD_BD_GRD4_RGN_CURSTT_MOD.to_csv(os.path.join(df1_fold, 'STD_BD_GRD4_RGN_CURSTT_MOD.csv'), index=False)

## 4등급 차량현황(그룹)
- 연도, 월, 시도, 시군구, 연료, 차종, 차종유형, 용도

### 현재 차량 대수

In [220]:
dfm.columns

Index(['자동차등록번호', '차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호',
       '차량연식', '차종', '용도', '최초등록일자', '제작일자', '차량관리번호', '연료', '차종유형', '제작사명',
       '차명', '자동차형식', '엔진형식', '저감장치구분', '저감장치종류', 'DPF_YN', '시도', '시군구',
       '조기폐차상태코드', '조기폐차최종승인YN', '조기폐차신청여부', '법정동코드_mod', '최초등록일자_년',
       '최초등록일자_월', '최초등록일자_일', '시군구_수정'],
      dtype='object')

In [231]:
num_car_by_local1 = dfm.groupby(['시도', '시군구_수정', '연료', '차종', '차종유형', '용도'], dropna=False, as_index=False).agg({'차대번호':'count', 'DPF_YN':'count', '조기폐차최종승인YN':'count'})
num_car_by_local1 = num_car_by_local1.rename(columns={'차대번호':'차량대수', 'DPF_YN':'저감장치부착대수', '조기폐차최종승인YN':'조기폐차대수'})
num_car_by_local1

,시도,시군구_수정,연료,차종,차종유형,용도,차량대수,저감장치부착대수,조기폐차대수
0,강원도,강릉시,LPG(액화석유가스),승용,대형,개인용,2,0,0
1,강원도,강릉시,LPG(액화석유가스),승용,중형,개인용,18,0,0
2,강원도,강릉시,LPG(액화석유가스),승용,중형,영업용,3,0,0
3,강원도,강릉시,LPG(액화석유가스),승합,경형,개인용,3,0,0
4,강원도,강릉시,LPG(액화석유가스),승합,소형,개인용,1,0,0
...,...,...,...,...,...,...,...,...,...
9169,충청북도,충주시,휘발유,승합,경형,개인용,13,0,0
9170,충청북도,충주시,휘발유,승합,소형,개인용,3,0,0
9171,충청북도,충주시,휘발유,승합,중형,개인용,2,0,0
9172,충청북도,충주시,휘발유,화물,경형,개인용,8,0,0


In [232]:
num_car_by_local1['저감장치미부착대수'] = num_car_by_local1['차량대수'] - num_car_by_local1['저감장치부착대수']
num_car_by_local1

,시도,시군구_수정,연료,차종,차종유형,용도,차량대수,저감장치부착대수,조기폐차대수,저감장치미부착대수
0,강원도,강릉시,LPG(액화석유가스),승용,대형,개인용,2,0,0,2
1,강원도,강릉시,LPG(액화석유가스),승용,중형,개인용,18,0,0,18
2,강원도,강릉시,LPG(액화석유가스),승용,중형,영업용,3,0,0,3
3,강원도,강릉시,LPG(액화석유가스),승합,경형,개인용,3,0,0,3
4,강원도,강릉시,LPG(액화석유가스),승합,소형,개인용,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
9169,충청북도,충주시,휘발유,승합,경형,개인용,13,0,0,13
9170,충청북도,충주시,휘발유,승합,소형,개인용,3,0,0,3
9171,충청북도,충주시,휘발유,승합,중형,개인용,2,0,0,2
9172,충청북도,충주시,휘발유,화물,경형,개인용,8,0,0,8


In [233]:
max_date = dfm['최초등록일자'].max()
max_year = max_date[:4]
max_month = max_date[4:6]
max_year, max_month
num_car_by_local1[['연도', '월']] = [max_year, max_month]
num_car_by_local1

,시도,시군구_수정,연료,차종,차종유형,용도,차량대수,저감장치부착대수,조기폐차대수,저감장치미부착대수,연도,월
0,강원도,강릉시,LPG(액화석유가스),승용,대형,개인용,2,0,0,2,2022,06
1,강원도,강릉시,LPG(액화석유가스),승용,중형,개인용,18,0,0,18,2022,06
2,강원도,강릉시,LPG(액화석유가스),승용,중형,영업용,3,0,0,3,2022,06
3,강원도,강릉시,LPG(액화석유가스),승합,경형,개인용,3,0,0,3,2022,06
4,강원도,강릉시,LPG(액화석유가스),승합,소형,개인용,1,0,0,1,2022,06
...,...,...,...,...,...,...,...,...,...,...,...,...
9169,충청북도,충주시,휘발유,승합,경형,개인용,13,0,0,13,2022,06
9170,충청북도,충주시,휘발유,승합,소형,개인용,3,0,0,3,2022,06
9171,충청북도,충주시,휘발유,승합,중형,개인용,2,0,0,2,2022,06
9172,충청북도,충주시,휘발유,화물,경형,개인용,8,0,0,8,2022,06


In [234]:
len(num_car_by_local1['월'].unique())

1

In [235]:
if len(num_car_by_local1['월'].unique()) != 1:
    # 오름차순 정렬
    num_car_by_local1 = num_car_by_local1.sort_values(['시도', '시군구_수정', '연료', '차종', '차종유형', '용도', '연도', '월']).reset_index(drop=True)
    num_car_by_local1['차량대수_전월'] = num_car_by_local1['차량대수'].shift(1)
    num_car_by_local1['감소율'] = (num_car_by_local1['차량대수_전월'] - num_car_by_local1['차량대수']) / num_car_by_local1['차량대수_전월']
    for n in range(num_car_by_local1.shape[0] // len(num_car_by_local1['월'].unique())):
        num_car_by_local1.loc[n*3, '감소율'] = np.nan
else:
    num_car_by_local1['감소율'] = np.nan

In [236]:
num_car_by_local1

,시도,시군구_수정,연료,차종,차종유형,용도,차량대수,저감장치부착대수,조기폐차대수,저감장치미부착대수,연도,월,감소율
0,강원도,강릉시,LPG(액화석유가스),승용,대형,개인용,2,0,0,2,2022,06,NaN
1,강원도,강릉시,LPG(액화석유가스),승용,중형,개인용,18,0,0,18,2022,06,NaN
2,강원도,강릉시,LPG(액화석유가스),승용,중형,영업용,3,0,0,3,2022,06,NaN
3,강원도,강릉시,LPG(액화석유가스),승합,경형,개인용,3,0,0,3,2022,06,NaN
4,강원도,강릉시,LPG(액화석유가스),승합,소형,개인용,1,0,0,1,2022,06,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9169,충청북도,충주시,휘발유,승합,경형,개인용,13,0,0,13,2022,06,NaN
9170,충청북도,충주시,휘발유,승합,소형,개인용,3,0,0,3,2022,06,NaN
9171,충청북도,충주시,휘발유,승합,중형,개인용,2,0,0,2,2022,06,NaN
9172,충청북도,충주시,휘발유,화물,경형,개인용,8,0,0,8,2022,06,NaN


In [237]:
num_car_by_local1['테이블생성일자'] = today_date

In [238]:
list(num_car_by_local1.columns)

['시도',
 '시군구_수정',
 '연료',
 '차종',
 '차종유형',
 '용도',
 '차량대수',
 '저감장치부착대수',
 '조기폐차대수',
 '저감장치미부착대수',
 '연도',
 '월',
 '감소율',
 '테이블생성일자']

In [248]:
STD_BD_DAT_GRD4_CAR_CURSTT = num_car_by_local1[[
    '연도',
    '월',
    '시도',
    '시군구_수정',
    '연료',
    '차종',
    '차종유형', 
    '용도',
    '차량대수',
    '감소율',
    '저감장치부착대수',
    '저감장치미부착대수',
    '조기폐차대수',
    '테이블생성일자',
]]

In [249]:
chc_col = {
    '연도':'YR',
    '월':'MM',
    '시도':'CTPV',
    '시군구_수정':'SGG',
    '연료':'FUEL_CD',
    '차종':'VHCTY_CD',
    '차종유형':'VHCTY_TY', 
    '용도':'PURPS_CD2',
    '차량대수':'VHCL_MKCNT',
    '감소율':'DEC_RT',
    '저감장치부착대수':'RDCDVC_EXTRNS_MKCNT',
    '저감장치미부착대수':'RDCDVC_UNAT_MKCNT',
    '조기폐차대수':'ELPDSRC_MKCNT',
    '테이블생성일자':'LOAD_DT', 
}

In [250]:
STD_BD_DAT_GRD4_CAR_CURSTT = STD_BD_DAT_GRD4_CAR_CURSTT.rename(columns=chc_col)
STD_BD_DAT_GRD4_CAR_CURSTT.columns

Index(['YR', 'MM', 'CTPV', 'SGG', 'FUEL_CD', 'VHCTY_CD', 'VHCTY_TY',
       'PURPS_CD2', 'VHCL_MKCNT', 'DEC_RT', 'RDCDVC_EXTRNS_MKCNT',
       'RDCDVC_UNAT_MKCNT', 'ELPDSRC_MKCNT', 'LOAD_DT'],
      dtype='object')

In [251]:
list(STD_BD_DAT_GRD4_CAR_CURSTT.columns)

['YR',
 'MM',
 'CTPV',
 'SGG',
 'FUEL_CD',
 'VHCTY_CD',
 'VHCTY_TY',
 'PURPS_CD2',
 'VHCL_MKCNT',
 'DEC_RT',
 'RDCDVC_EXTRNS_MKCNT',
 'RDCDVC_UNAT_MKCNT',
 'ELPDSRC_MKCNT',
 'LOAD_DT']

### [출력] STD_BD_DAT_GRD4_CAR_CURSTT

In [ ]:
# expdf = STD_BD_DAT_GRD4_CAR_CURSTT
# table_nm = 'STD_BD_DAT_GRD4_CAR_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

In [252]:
# # 0s
# STD_BD_DAT_GRD4_CAR_CURSTT.to_csv(os.path.join(df1_fold, 'STD_BD_DAT_GRD4_CAR_CURSTT.csv'), index=False)

# code end